In [7]:
import tusz_subset
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Activation, Conv2D

In [8]:
def simple_binary_classifier(nb_classes, channels=64, samples=128) -> Model:
    input_layer = Input(shape=(channels, samples, 1))
    flatten = Flatten(name='flatten')(input_layer)
    hidden = Dense(nb_classes, name='hidden')(flatten)
    output = Dense(nb_classes, name='output')(hidden)
    softmax = Activation('softmax', name='softmax')(output)
    return Model(inputs=input_layer, outputs=softmax)

In [9]:
(x_train, y_train), (x_dev, y_dev) = tfds.as_numpy(tfds.load('tusz_subset',
                                                             split=['train', 'dev'],
                                                             batch_size=-1,
                                                             as_supervised=True))

In [10]:
tfds.benchmark(x_dev, batch_size=32)



************ Summary ************

Examples/sec (First included) 22035496.13 ex/sec (total: 1447232 ex, 0.07 sec)
Examples/sec (First only) 4184647.28 ex/sec (total: 32 ex, 0.00 sec)
Examples/sec (First excluded) 22037574.80 ex/sec (total: 1447200 ex, 0.07 sec)


,duration,num_examples,avg
first+lasts,0.065677,1447232,2.203550e+07
first,0.000008,32,4.184647e+06
lasts,0.065670,1447200,2.203757e+07


In [15]:
import numpy as np
for item in zip(x_train, y_train):
    print(np.shape(x_train))
    print(y_train)
    break

(9666, 64, 128, 1)
[0 0 0 ... 0 1 0]


In [12]:
model = simple_binary_classifier(2)

In [64]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
             )

In [67]:
print('Fit model on training data')
history = model.fit(
    x_train,
    y_train,
    epochs=6,
    validation_data=(x_dev, y_dev),
)
history.history

Fit model on training data
Epoch 1/6
303/303 [==============================] - 2s 8ms/step - loss: 0.4644 - sparse_categorical_accuracy: 0.8227 - val_loss: 0.4999 - val_sparse_categorical_accuracy: 0.8219
Epoch 2/6
303/303 [==============================] - 1s 4ms/step - loss: 0.4642 - sparse_categorical_accuracy: 0.8226 - val_loss: 0.5010 - val_sparse_categorical_accuracy: 0.8219
Epoch 3/6
303/303 [==============================] - 1s 4ms/step - loss: 0.4640 - sparse_categorical_accuracy: 0.8226 - val_loss: 0.5022 - val_sparse_categorical_accuracy: 0.8219
Epoch 4/6
303/303 [==============================] - 1s 4ms/step - loss: 0.4636 - sparse_categorical_accuracy: 0.8226 - val_loss: 0.5029 - val_sparse_categorical_accuracy: 0.8218
Epoch 5/6
303/303 [==============================] - 1s 4ms/step - loss: 0.4635 - sparse_categorical_accuracy: 0.8225 - val_loss: 0.5049 - val_sparse_categorical_accuracy: 0.8218
Epoch 6/6
303/303 [==============================] - 1s 4ms/step - loss: 0.463

{'loss': [0.4644293189048767,
  0.4641708433628082,
  0.4639703333377838,
  0.4636498689651489,
  0.463479220867157,
  0.46313977241516113],
 'sparse_categorical_accuracy': [0.8226774334907532,
  0.8225739598274231,
  0.8225739598274231,
  0.8225739598274231,
  0.822470486164093,
  0.822470486164093],
 'val_loss': [0.4998953938484192,
  0.5009856224060059,
  0.502238929271698,
  0.5028719305992126,
  0.5049048066139221,
  0.5070930123329163],
 'val_sparse_categorical_accuracy': [0.8219165802001953,
  0.821872353553772,
  0.8218502402305603,
  0.8218281269073486,
  0.821806013584137,
  0.8217618465423584]}

In [68]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_dev, y_dev, batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data
354/354 [==============================] - 1s 2ms/step - loss: 0.5071 - sparse_categorical_accuracy: 0.8218
test loss, test acc: [0.5070934295654297, 0.8217618465423584]


In [69]:
# Save model weights!
import os
model.save(os.getcwd())

INFO:tensorflow:Assets written to: /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/assets


INFO:tensorflow:Assets written to: /Users/robert/Desktop/School/Grad/481c/seizure-classification/training/assets


In [83]:
1 - (sum(y_train) / len(y_train))

0.8225739706186633

In [ ]:
# Model learns nothing!!
# Try to add conv layer

In [126]:
input_shape = (64, 128, 1)
def cnn_binary_classifier(nb_classes, channels=64, samples=128) -> Model:
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(nb_classes))
    return model

In [127]:
# train conv model
cnn_model = cnn_binary_classifier(2)

In [128]:
cnn_model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
                 )

In [129]:
print('Fit model on training data')
history = cnn_model.fit(
    x_train,
    y_train,
    epochs=6,
    validation_data=(x_dev, y_dev),
)
history.history


Fit model on training data
Epoch 1/6
303/303 [==============================] - 67s 221ms/step - loss: 0.4966 - sparse_categorical_accuracy: 0.8217 - val_loss: 0.4634 - val_sparse_categorical_accuracy: 0.8252
Epoch 2/6
303/303 [==============================] - 63s 209ms/step - loss: 0.4694 - sparse_categorical_accuracy: 0.8226 - val_loss: 0.4665 - val_sparse_categorical_accuracy: 0.8252
Epoch 3/6
303/303 [==============================] - 69s 227ms/step - loss: 0.4687 - sparse_categorical_accuracy: 0.8226 - val_loss: 0.4634 - val_sparse_categorical_accuracy: 0.8252
Epoch 4/6
302/303 [============================>.] - ETA: 0s - loss: 0.4683 - sparse_categorical_accuracy: 0.8225

KeyboardInterrupt: 

In [130]:
from torch import nn
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm


def EEGNet(nb_classes, channels=64, samples=128,
           dropout_rate=0.5, kernel_length=64, f1=8,
           d=2, f2=16, norm_rate=0.25, dropout_type='Dropout'):
    """ Keras Implementation of EEGNet
    http://iopscience.iop.org/article/10.1088/1741-2552/aace8c/meta
    Note that this implements the newest version of EEGNet and NOT the earlier
    version (version v1 and v2 on arxiv). We strongly recommend using this
    architecture as it performs much better and has nicer properties than
    our earlier version. For example:

        1. Depth-wise Convolutions to learn spatial filters within a
        temporal convolution. The use of the depth_multiplier option maps
        exactly to the number of spatial filters learned within a temporal
        filter. This matches the setup of algorithms like FBCSP which learn
        spatial filters within each filter in a filter-bank. This also limits
        the number of free parameters to fit when compared to a fully-connected
        convolution.

        2. Separable Convolutions to learn how to optimally combine spatial
        filters across temporal bands. Separable Convolutions are Depth-wise
        Convolutions followed by (1x1) Point-wise Convolutions.


    While the original paper used Dropout, we found that SpatialDropout2D
    sometimes produced slightly better results for classification of ERP
    signals. However, SpatialDropout2D significantly reduced performance
    on the Oscillatory dataset (SMR, BCI-IV Dataset 2A). We recommend using
    the default Dropout in most cases.

    Assumes the input signal is sampled at 128Hz. If you want to use this model
    for any other sampling rate you will need to modify the lengths of temporal
    kernels and average pooling size in blocks 1 and 2 as needed (double the
    kernel lengths for double the sampling rate, etc). Note that we haven't
    tested the model performance with this rule so this may not work well.

    The model with default parameters gives the EEGNet-8,2 model as discussed
    in the paper. This model should do pretty well in general, although it is
    advised to do some model searching to get optimal performance on your
    particular dataset. We set f2 = f1 * d (number of input filters = number of output filters)
    for the SeparableConv2D layer. We haven't extensively tested other values of this
    parameter (say, f2 < f1 * d for compressed learning, and f2 > f1 * d for
    overcomplete). We believe the main parameters to focus on are f1 and d.
    Inputs:

      nb_classes      : int, number of classes to classify
      channels, samples  : number of channels and time points in the EEG data
      dropout_rate     : dropout fraction
      kernel_length      : length of temporal convolution in first layer. We found
                        that setting this to be half the sampling rate worked
                        well in practice. For the SMR dataset in particular
                        since the data was high-passed at 4Hz we used a kernel
                        length of 32.
      f1, f2          : number of temporal filters (f1) and number of point-wise
                        filters (f2) to learn. Default: f1 = 8, f2 = f1 * d.
      d               : number of spatial filters to learn within each temporal
                        convolution. Default: d = 2
      dropout_type     : Either SpatialDropout2D or Dropout, passed as a string.
    """

    if dropout_type == 'SpatialDropout2D':
        dropout_type = SpatialDropout2D
    elif dropout_type == 'Dropout':
        dropout_type = Dropout
    else:
        raise ValueError('dropout_type must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')

    input1 = Input(shape=(channels, samples, 1))

    ##################################################################
    block1 = Conv2D(f1, (1, kernel_length), padding='same',
                    input_shape=(channels, samples, 1),
                    use_bias=False)(input1)
    block1 = BatchNormalization()(block1)
    block1 = DepthwiseConv2D((channels, 1), use_bias=False,
                             depth_multiplier=d,
                             depthwise_constraint=max_norm(1.))(block1)
    block1 = BatchNormalization()(block1)
    block1 = Activation('elu')(block1)
    block1 = AveragePooling2D((1, 4))(block1)
    block1 = dropout_type(dropout_rate)(block1)

    block2 = SeparableConv2D(f2, (1, 16),
                             use_bias=False, padding='same')(block1)
    block2 = BatchNormalization()(block2)
    block2 = Activation('elu')(block2)
    block2 = AveragePooling2D((1, 8))(block2)
    block2 = dropout_type(dropout_rate)(block2)

    flatten = Flatten(name='flatten')(block2)

    dense = Dense(nb_classes, name='dense',
                  kernel_constraint=max_norm(norm_rate))(flatten)
    softmax = Activation('softmax', name='softmax')(dense)

    return Model(inputs=input1, outputs=softmax)

In [134]:
# train conv model
eeg_net = EEGNet(2)

In [135]:
eeg_net.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
               )

In [136]:
print('Fit model on training data')
history = eeg_net.fit(
    x_train,
    y_train,
    epochs=6,
    validation_data=(x_dev, y_dev),
)
history.history


Fit model on training data
Epoch 1/6
303/303 [==============================] - 61s 198ms/step - loss: 0.4790 - sparse_categorical_accuracy: 0.8165 - val_loss: 0.5260 - val_sparse_categorical_accuracy: 0.8229
Epoch 2/6
303/303 [==============================] - 49s 162ms/step - loss: 0.4225 - sparse_categorical_accuracy: 0.8207 - val_loss: 0.4957 - val_sparse_categorical_accuracy: 0.8003
Epoch 3/6
303/303 [==============================] - 47s 156ms/step - loss: 0.4135 - sparse_categorical_accuracy: 0.8215 - val_loss: 0.6550 - val_sparse_categorical_accuracy: 0.7628
Epoch 4/6
303/303 [==============================] - 48s 158ms/step - loss: 0.4058 - sparse_categorical_accuracy: 0.8202 - val_loss: 1.0603 - val_sparse_categorical_accuracy: 0.3896
Epoch 5/6
303/303 [==============================] - 47s 156ms/step - loss: 0.4000 - sparse_categorical_accuracy: 0.8186 - val_loss: 1.2473 - val_sparse_categorical_accuracy: 0.7987
Epoch 6/6
303/303 [==============================] - 52s 173ms/

{'loss': [0.4790300130844116,
  0.42250344157218933,
  0.41348060965538025,
  0.4058060050010681,
  0.40003502368927,
  0.3971465826034546],
 'sparse_categorical_accuracy': [0.8164700865745544,
  0.8207117915153503,
  0.8215394020080566,
  0.8201944828033447,
  0.8186426758766174,
  0.8214359879493713],
 'val_loss': [0.5260092616081238,
  0.49569565057754517,
  0.6549774408340454,
  1.0602548122406006,
  1.2473222017288208,
  0.59682697057724],
 'val_sparse_categorical_accuracy': [0.8228673934936523,
  0.8002697825431824,
  0.7627913355827332,
  0.389643132686615,
  0.798721969127655,
  0.6695927381515503]}

In [137]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = eeg_net.evaluate(x_dev, y_dev, batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data
354/354 [==============================] - 25s 70ms/step - loss: 0.5968 - sparse_categorical_accuracy: 0.6696
test loss, test acc: [0.5968274474143982, 0.6695927381515503]


In [ ]:
# Try models from article sent by raj

In [31]:
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler

In [34]:
# load the model, columns, and scaler
best_model = pickle.load(open('best_classifier.pkl','rb'))
cols_input = pickle.load(open('cols_input.sav','rb'))
scaler = pickle.load(open('scaler.sav', 'rb'))

/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator FunctionTransformer from version 0.20.3 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator FeatureUnion from version 0.20.3 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/anaconda3/lib/python3.8/site-packages/

In [35]:
# load the data
df_train = pd.read_csv('df_train.csv')
df_valid= pd.read_csv('df_valid.csv')
df_test= pd.read_csv('df_test.csv')

# create X and y matrices
X_train = df_train[cols_input].values
X_valid = df_valid[cols_input].values
X_test = df_test[cols_input].values

y_train = df_train['OUTPUT_LABEL'].values
y_valid = df_valid['OUTPUT_LABEL'].values
y_test = df_test['OUTPUT_LABEL'].values

# transform our data matrices 
X_train_tf = scaler.transform(X_train)
X_valid_tf = scaler.transform(X_valid)
X_test_tf = scaler.transform(X_test)

RuntimeError: The reset parameter is False but there is no n_features_in_ attribute. Is this estimator fitted?